In [447]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import random
from datetime import datetime
import math
import matplotlib.pyplot as plt
from flask import jsonify
import sqlite3
import time


In [360]:
#setup splinter
executable_path= {'executable_path': ChromeDriverManager().install()}
browser= Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Get LATEST driver version for 98.0.4758
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/juliankauffmann/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


In [18]:
#####CREATE WEB-SCRAPING BOT##########

In [104]:
#opens flix
def open_flix(the_url):
    browser.visit(the_url)

In [24]:
#return updated html
def get_updated_html():
    thesoup = BeautifulSoup(browser.html, 'html.parser')

    return thesoup

In [445]:
demographics_df=pd.DataFrame({"title":[],"view":[],"category":[],
              "country":[],"premiere":[], "genre":[],"generation":[],"gender":[],"viewing_country":[]})

generations=['generation-z','baby-boomers','generation-y','generation-x']
genders=['men','women']
countries=['brazil','canada','france','germany','india','mexico','united-kingdom','united-states']

for gender in genders:
    for country in countries:
        for generation in generations:
            the_url=f'https://flixpatrol.com/demographics/{generation}/{gender}/{country}/'
            open_flix(the_url)
            time.sleep(3)
            temp_df=pull_flix_demo(generation,gender,country)
            demographics_df=pd.concat([demographics_df,temp_df])
                
demographics_df.head()

,title,view,category,country,premiere,genre,generation,gender,viewing_country
0,Friends,"7,700,000",TV Show,United States,09/22/1994,|Comedy|Sitcom,generation-z,men,brazil
1,Rio,"1,800,000",Movie,United States,04/15/2011,|Animated|For Girls,generation-z,men,brazil
2,The Simpsons,"1,300,000",TV Show,United States,12/17/1989,|Animation|Adult Animation,generation-z,men,brazil
3,Supernatural,"1,200,000",TV Show,United States,09/13/2005,|Fantasy|Supernatural,generation-z,men,brazil
4,Vikings,"1,100,000",TV Show,Ireland,03/03/2013,|History|Vikings,generation-z,men,brazil


In [454]:
connection = sqlite3.connect('flix.db')
cursor=connection.cursor()
demographics_df.shape

(11291, 9)

In [455]:
demographics_df.to_sql("flix_shows", connection, if_exists='fail',index=False)

In [443]:
def pull_flix_demo(generation,gender,country):
    titles=[]
    the_html=get_updated_html()
    temp_titles=the_html.find_all("div", class_="group-hover:underline")

    for title in temp_titles:
        titles.append(title.text.split('\n')[1].split('\t')[9])
        #print(title.text)

    #get views of top 100
    views=[]

    temp_views=the_html.find_all("td", class_="table-td text-right text-gray-400 font-semibold tabular-nums")

    for view in temp_views:
        views.append(view.text)

    #get category of top 100
    categories=[]

    temp_cats=the_html.find_all("div", class_="flex flex-wrap text-sm leading-6 text-gray-500")

    for cat in temp_cats:
        for c in cat.span:
            categories.append(c)

    #get country, premiere date, and genre info of top 100
    countries=[]
    premieres=[]
    genres=[]

    temp_infos=the_html.find_all("div", class_="flex flex-wrap text-sm leading-6 text-gray-500")

    for info in temp_infos:
        temp_genre=""
        for i in range(0,len(info.text.split('\n'))-1):
            temp=info.text.split('\n')[i]       
            if i>6:
                if (temp !="|") and (temp !=""):
                    temp_genre=temp_genre+'|'+temp            

        genres.append(temp_genre)
        premieres.append(info.text.split('\n')[5])
        countries.append(info.text.split('\n')[3])

    return pd.DataFrame({"title":titles,"view":views,"category":categories,
                         "country":countries,"premiere":premieres, "genre":genres,
                         "generation":generation,"gender":gender,"viewing_country":country})
    

In [418]:
flix_feb12_genZ

,title,view,category,country,premiere,genre
0,Friends,"7,700,000",TV Show,United States,09/22/1994,|Comedy|Sitcom
1,Rio,"1,800,000",Movie,United States,04/15/2011,|Animated|For Girls
2,The Simpsons,"1,300,000",TV Show,United States,12/17/1989,|Animation|Adult Animation
3,Supernatural,"1,200,000",TV Show,United States,09/13/2005,|Fantasy|Supernatural
4,Vikings,"1,100,000",TV Show,Ireland,03/03/2013,|History|Vikings
...,...,...,...,...,...,...
137,The Martian,"1,000",Movie,United States,10/02/2015,|Science Fiction|Space
138,We're the Millers,"1,000",Movie,United States,08/07/2013,|Comedy|Situation
139,E.T. the Extra-Terrestrial,"1,000",Movie,United States,04/03/1982,|Science Fiction|Aliens
140,American Sniper,"1,000",Movie,United States,12/25/2014,|War|Iraq War


In [274]:
#set up sqlite
connection = sqlite3.connect('flix_shows.db')
cursor=connection.cursor()

In [275]:
flix_feb12_genZ.to_sql("flix_genz", connection, if_exists='fail',index=False)

In [278]:
pd.read_sql_query("select * from flix_genz",connection)

,title,view,category,country,premiere,genre
0,Friends,"78,199,300",TV Show,United States,09/22/1994,|Comedy|Sitcom
1,Supernatural,"20,568,700",TV Show,United States,09/13/2005,|Fantasy|Supernatural
2,Tom and Jerry,"17,723,400",TV Show,United States,02/10/1940,|Animation|For Older Kids
3,Game of Thrones,"15,103,500",TV Show,United States,04/17/2011,| HBO|Fantasy|Dragons
4,The Simpsons,"14,288,200",TV Show,United States,12/17/1989,|Animation|Adult Animation
...,...,...,...,...,...,...
455,Best Exotic Marigold Hotel,"8,000",Franchise,United States,2011 – 2015,|Comedy
456,All the King's Men,"7,000",Movie,United States,09/10/2006,|Drama
457,The Constant Gardener,"6,000",Movie,United Kingdom,08/31/2005,|Thriller|Crime Drama
458,Star Trek: Alternate Reality,"4,600",Franchise,United States,2009 – 2016,|Science Fiction|Space opera


In [254]:
browser.quit()